In [1]:
from kaggle_secrets import UserSecretsClient
import wandb
from huggingface_hub import login
user_secrets = UserSecretsClient()

login(token = user_secrets.get_secret("HUGGING_FACE_API_KEY"))
wandb.login(key=user_secrets.get_secret("wandb_api_key"))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
import json
import os

def split_json(path, train_ratio=0.7, val_ratio=0.15):
    # Load the original JSON file
    with open(path, 'rb') as f:
        data = json.load(f)

    # Calculate the indices for splitting
    train_end = int(train_ratio * len(data['data']))
    val_end = int(val_ratio * len(data['data'])) + train_end

    # Split the data
    train_data = data['data'][:train_end]
    val_data = data['data'][train_end:val_end]
    test_data = data['data'][val_end:]

    # Get the base name of the original file
    base_name = os.path.basename(path).split('.')[0]

    # Save the split data into new JSON files
    with open(f'{base_name}_train.json', 'w') as f:
        json.dump({'data': train_data}, f)
    with open(f'{base_name}_val.json', 'w') as f:
        json.dump({'data': val_data}, f)
    with open(f'{base_name}_test.json', 'w') as f:
        json.dump({'data': test_data}, f)

# Call the function
split_json('/kaggle/input/covid-qa/COVID-QA.json')


In [3]:
import json

def read_json(paths):
    # initialize lists for contexts, questions, and answers
    contexts = []
    questions = []
    answers = []
    ids = []

    for path in paths:
        with open(path, 'rb') as f:
            squad_dict = json.load(f)

        # iterate through all data in squad data
        for group in squad_dict['data']:
            for passage in group['paragraphs']:
                context = passage['context']
                for qa in passage['qas']:
                    question = qa['question']
                    id = qa['id']
                    if 'plausible_answers' in qa.keys():
                        access = 'plausible_answers'
                    else:
                        access = 'answers'
                    for answer in qa['answers']:
                        # append data to lists
                        contexts.append(context)
                        questions.append(question)
                        answers.append(answer)
                        ids.append(id)
    # return formatted data lists
    return contexts, questions, answers, ids

train_path = ['/kaggle/input/mashqa-dataset/train_webmd_squad_v2_consec.json',
             '/kaggle/working/COVID-QA_train.json']

val_path = ['/kaggle/input/mashqa-dataset/val_webmd_squad_v2_consec.json',
           '/kaggle/working/COVID-QA_val.json']

test_path_mash = ['/kaggle/input/mashqa-dataset/test_webmd_squad_v2_consec.json']

test_path_covid = ['/kaggle/working/COVID-QA_test.json']

# apply function
train_contexts, train_questions, train_answers, train_ids = read_json(train_path)
val_contexts, val_questions, val_answers, val_ids = read_json(val_path)
test_contexts_mash, test_questions_mash, test_answers_mash, test_ids_mash = read_json(test_path_mash)
test_contexts_covid, test_questions_covid, test_answers_covid, test_ids_covid = read_json(test_path_covid)

### Tokenize

The idea is to use a custom dataset, so we need to make sure that it has the same structure as squad from Hugging Face dataset

In [4]:
def create_answers_dict(x):
    dict_ = {"text":[(x["answer"])], "answer_start":[(int(x["answer_start"]))]}
    return dict_



In [5]:
import pandas as pd

#train
contexts_df_train = pd.DataFrame(train_contexts, columns=['context'])
questions_df_train = pd.DataFrame(train_questions, columns=['question'])
ids_df_train = pd.DataFrame(train_ids, columns=['id'])
answers_df_train = pd.DataFrame.from_records(train_answers)
df_train = contexts_df_train.copy()
df_train["question"] = questions_df_train["question"]
df_train["answer"] = answers_df_train["text"]
df_train["answer_start"] = answers_df_train["answer_start"]
df_train["id"] = ids_df_train['id']
df_train["answers"] = df_train.apply(lambda x: create_answers_dict(x), axis = 1)


# val
ids_df_val = pd.DataFrame(val_ids, columns=['id'])  # Add this line
contexts_df_val = pd.DataFrame(val_contexts, columns=['context'])
questions_df_val = pd.DataFrame(val_questions, columns=['question'])
answers_df_val = pd.DataFrame.from_records(val_answers)
df_val = contexts_df_val.copy()
df_val["question"] = questions_df_val["question"]
df_val["answer"] = answers_df_val["text"]
df_val["answer_start"] = answers_df_val["answer_start"]
df_val["id"] = ids_df_val['id']  # Add this line
df_val["answers"] = df_val.apply(lambda x: create_answers_dict(x), axis = 1)

# test_mash
ids_df_test_mash = pd.DataFrame(test_ids_mash, columns=['id'])
contexts_df_test_mash = pd.DataFrame(test_contexts_mash, columns=['context'])
questions_df_test_mash = pd.DataFrame(test_questions_mash, columns=['question'])
answers_df_test_mash = pd.DataFrame.from_records(test_answers_mash)

df_test_mash = contexts_df_test_mash.copy()
df_test_mash["question"] = questions_df_test_mash["question"]
df_test_mash["answer"] = answers_df_test_mash["text"]
df_test_mash["answer_start"] = answers_df_test_mash["answer_start"]
df_test_mash["id"] = ids_df_test_mash['id']
df_test_mash["answers"] = df_test_mash.apply(lambda x: create_answers_dict(x), axis = 1)




# test_covid
ids_df_test_covid = pd.DataFrame(test_ids_covid, columns=['id'])
contexts_df_test_covid = pd.DataFrame(test_contexts_covid, columns=['context'])
questions_df_test_covid = pd.DataFrame(test_questions_covid, columns=['question'])
answers_df_test_covid = pd.DataFrame.from_records(test_answers_covid)

df_test_covid = contexts_df_test_covid.copy()
df_test_covid["question"] = questions_df_test_covid["question"]
df_test_covid["answer"] = answers_df_test_covid["text"]
df_test_covid["answer_start"] = answers_df_test_covid["answer_start"]
df_test_covid["id"] = ids_df_test_covid['id']
df_test_covid["answers"] = df_test_covid.apply(lambda x: create_answers_dict(x), axis = 1)

Saving dataframes to .csv

In [6]:

df_train.sample(frac = 0.5)[['id', 'context', 'question', 'answers']].to_csv('/kaggle/working/dataset_train.csv', index=False)
df_val.sample(frac = 0.5)[['id', 'context', 'question', 'answers']].to_csv('/kaggle/working/dataset_val.csv', index=False)
df_test_mash.sample(frac = 0.5)[['id', 'context', 'question', 'answers']].to_csv('/kaggle/working/dataset_test_mash.csv', index=False)
df_test_covid.sample(frac = 0.5)[['id', 'context', 'question', 'answers']].to_csv('/kaggle/working/dataset_test_covid.csv', index=False)

In [7]:
from datasets import load_dataset

data_files = {"train": "/kaggle/working/dataset_train.csv", 
              "validation": "/kaggle/working/dataset_val.csv", 
              "test_mash": "/kaggle/working/dataset_test_mash.csv",
              "test_covid": "/kaggle/working/dataset_test_covid.csv"}
ds = load_dataset("csv", data_files=data_files)
ds

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test_mash split: 0 examples [00:00, ? examples/s]

Generating test_covid split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10500
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1590
    })
    test_mash: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1307
    })
    test_covid: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 257
    })
})

The column "answers" wasn't saved as a dict (python data structure), it's a string. We nee to change from that string containing the dict to a real dict

In [8]:
def convert_text(batch):
  aux_list = []
  for x, y in zip(batch["answers"], batch["answers"]):
    my_dict = {"text":eval(x)["text"], "answer_start":eval(x)["answer_start"]}
    aux_list.append(my_dict)
    
  return {"texts":aux_list}



prepared_ds = ds.map(convert_text, batched=True)
prepared_ds = prepared_ds.remove_columns("answers")
prepared_ds = prepared_ds.rename_column("texts", "answers")

Map:   0%|          | 0/10500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

Map:   0%|          | 0/1307 [00:00<?, ? examples/s]

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoTokenizer
model_id = 'bert-base-cased'

tokenizer = AutoTokenizer.from_pretrained(model_id)
max_length = 512
stride = 128

def preprocess_training(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [10]:
train_dataset = prepared_ds["train"].map(
    preprocess_training,
    batched=True,
    remove_columns=prepared_ds["train"].column_names,
)
train_dataset

Map:   0%|          | 0/10500 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 44805
})

In [11]:

def preprocess_validation(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [12]:
validation_dataset = prepared_ds["validation"].map(
    preprocess_validation,
    batched=True,
    remove_columns=prepared_ds["validation"].column_names,
)

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

In [13]:
# import sys
# import io
# from contextlib import redirect_stdout

# def quiet_map(dataset, function, **kwargs):
#     # Create a string buffer
#     buffer = io.StringIO()

#     # Redirect the standard output to the buffer
#     with redirect_stdout(buffer):
#         # Call the map function
#         result = dataset.map(function, **kwargs)

#     # Return the result
#     return result

# # Now you can call the quiet_map function instead of the map function
# tokenized = quiet_map(prepared_ds, preprocess_function, batched=True, remove_columns=ds["train"].column_names)

In [14]:
# tokenized

### Training
Define training arguments and trainer

In [15]:
from transformers import DefaultDataCollator, AutoModelForQuestionAnswering, TrainingArguments, Trainer

output_model = 'bert-qa-mash-covid'
data_collator = DefaultDataCollator()

model = AutoModelForQuestionAnswering.from_pretrained(model_id)

training_args = TrainingArguments(
    output_dir= output_model,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    save_strategy="epoch"
)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized["train"],
#     eval_dataset=tokenized["validation"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


2024-06-12 16:14:09.745867: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 16:14:09.745972: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 16:14:09.850602: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [16]:
trainer.train()

wandb: Currently logged in as: thienlequang62ld (quangthien). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240612_161431-vqa6eboz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run clean-sunset-64
wandb: ⭐️ View project at https://wandb.ai/quangthien/huggingface
wandb: 🚀 View run at https://wandb.ai/quangthien/huggingface/runs/vqa6eboz


Epoch,Training Loss,Validation Loss
1,0.908100,No log
2,0.706000,No log
3,0.563300,No log


TrainOutput(global_step=8403, training_loss=0.7855498533965036, metrics={'train_runtime': 8029.7039, 'train_samples_per_second': 16.74, 'train_steps_per_second': 1.046, 'total_flos': 3.512220355666944e+16, 'train_loss': 0.7855498533965036, 'epoch': 3.0})

In [17]:
trainer.save_model()
# metrics = trainer.evaluate(tokenized["validation"])

kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "question-answering",
    "dataset": "mashqa_covid_dataset",
    "tags":["question-answering", "nlp"]
}

In [18]:
trainer.push_to_hub(commit_message = "model tuned", **kwargs)

CommitInfo(commit_url='https://huggingface.co/Eurosmart/bert-qa-mash-covid/commit/416721ce3b6993e273c1e237e2d8ca30b00fc94c', commit_message='model tuned', commit_description='', oid='416721ce3b6993e273c1e237e2d8ca30b00fc94c', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
from tqdm.auto import tqdm
import collections
import numpy as np
n_best = 20
max_answer_length = 100


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [20]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
import evaluate

metric = evaluate.load("squad")

predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, prepared_ds["validation"])

  0%|          | 0/1590 [00:00<?, ?it/s]

{'exact_match': 21.446540880503143, 'f1': 52.952550275165336}

In [22]:
# from evaluate import evaluator

# # Initialize the evaluator
# task_evaluator = evaluator("question-answering")

# # Compute the evaluation results
# results = task_evaluator.compute(
#     model_or_pipeline=f"Eurosmart/{output_model}",  # replace with your model
#     data=prepared_ds['test_mash'],  # replace with your dataset
# #     metric="squad",  # replace with your metric
# )

# # Print the results
# print(results)


In [23]:
# from evaluate import evaluator

# # Initialize the evaluator
# task_evaluator = evaluator("question-answering")

# # Compute the evaluation results
# results = task_evaluator.compute(
#     model_or_pipeline=f"Eurosmart/{output_model}",  # replace with your model
#     data=prepared_ds['test_covid'],  # replace with your dataset
# #     metric="squad",  # replace with your metric
# )

# # Print the results
# print(results)


In [24]:
# from evaluate import evaluator

# # Initialize the evaluator
# task_evaluator = evaluator("question-answering")

# # Compute the evaluation results
# results = task_evaluator.compute(
#     model_or_pipeline=f"Eurosmart/{output_model}",  # replace with your model
#     data=prepared_ds['validation'],  # replace with your dataset
# #     metric="squad",  # replace with your metric
# )

# # Print the results
# print(results)
